In [18]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
from datetime import datetime
import numpy as np

In [27]:


def wsj_query(stock_name, start_date, end_date):

    my_url = "https://www.wsj.com/market-data/quotes/HU/XBUD/{stock_name}/historical-prices/download?MOD_VIEW=page&num_rows=3768.9583333333335&range_days=3768.9583333333335&startDate={start_date}&endDate={end_date}".format(stock_name=stock_name, start_date=start_date, end_date=end_date)

    #open connection, grab page, close connection
    resp_obj = urlopen(Request(my_url,headers={'User-Agent': 'Mozilla/6.0'} ))
    page_html = resp_obj.read()
    resp_obj.close()

    #,then convert to format that pandas can read
    parsed_text = BeautifulSoup(page_html).body.p.text
    text_obj = StringIO(parsed_text)

    date_parser = lambda x: datetime.strptime(x, "%m/%d/%y").date()
    oneYearDF = pd.read_csv(text_obj,parse_dates=["Date"], date_parser = date_parser)
    
    return oneYearDF

def calculcateReturns(column):
    return column[1:].to_numpy() - column[:-1].to_numpy()

In [84]:
stock_names = pd.read_csv("name_source.csv",sep="\t")["Név"].str.replace(" ", "").to_list()
[ stock_names.remove(x) for x in ["APPENINN","AUTOWALLIS","CIGPANNONIA","DUNAHOUSE", "ETFBUXOTP", "GSPARK","MASTERPLAST","MTELEKOM","PANNERGY","RICHTER","WABERERS"] ]
[stock_names.append(x) for x in ["ETFB","APPEN","AUTOW","CIG","GSP","MAST","MTEL","PERGY","RICHT"]]


for stock_name in stock_names:
    years = np.arange(2010,2021)
    mainDF = pd.DataFrame()
    for year in years:
        oneYearDF = wsj_query(stock_name = stock_name, start_date = "01/01/{}".format(year), end_date = "12/31/{}".format(year))
        mainDF = pd.concat([mainDF, oneYearDF], ignore_index = True)
    mainDF.columns = mainDF.columns.str.replace(' ', '')
    mainDF.to_csv("data/historical/{}.csv".format(stock_name))

In [83]:
stock_names = pd.read_csv("name_source.csv",sep="\t")["Név"].str.replace(" ", "").to_list()
[ stock_names.remove(x) for x in ["APPENINN","AUTOWALLIS","CIGPANNONIA","DUNAHOUSE", "ETFBUXOTP", "GSPARK","MASTERPLAST","MTELEKOM","PANNERGY","RICHTER","WABERERS"] ]
[stock_names.append(x) for x in ["ETFB","APPEN","AUTOW","CIG","GSP","MAST","MTEL","PERGY","RICHT"]]
stock_names

['4IG',
 'ALTEO',
 'ANY',
 'BIF',
 'MOL',
 'OPUS',
 'OTP',
 'RABA',
 'TAKAREKJZB',
 'ZWACK',
 'ETFB',
 'APPEN',
 'AUTOW',
 'CIG',
 'GSP',
 'MAST',
 'MTEL',
 'PERGY',
 'RICHT']

0             4IG 
1           ALTEO 
2             ANY 
3        APPENINN 
4      AUTOWALLIS 
5             BIF 
6     CIGPANNONIA 
7       DUNAHOUSE 
8       ETFBUXOTP 
9          GSPARK 
10    MASTERPLAST 
11            MOL 
12       MTELEKOM 
13           OPUS 
14            OTP 
15       PANNERGY 
16           RABA 
17        RICHTER 
18     TAKAREKJZB 
19       WABERERS 
20          ZWACK 
Name: Név, dtype: object